### Analyse automatique du narratif 
`Shami THIRION SEN`

In [1]:
import numpy as np
import json
import glob
from pprint import pprint
from collections import defaultdict
import os, math
from utils import * # import des fonctions depuis le fichier utils

import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#visualisation
import pyLDAvis
import pyLDAvis.gensim

# outils TAL en bengali
import BnLemma as lm
from bnlp import BengaliPOS

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
banglaGanashakti = "../corpus/txtFiles/banglaGanashakti.txt"

bartamanpatrika = "../corpus/txtFiles/bartamanpatrika.txt"
anandabazar = "../corpus/txtFiles/anandabazar.txt"


In [3]:
# Bangla POS tagging
bn_pos = BengaliPOS()
from bnlp import BengaliCorpus as corpus
stopwords = corpus.stopwords[:20] + ['রি' + 'টি']

#### BanglaGanashakti 

In [4]:
filtered_ganashakti = read_corpus(banglaGanashakti, 1000)
filtered_bartamanpatrika = read_corpus(bartamanpatrika, 1000)
filtered_anandabazar = read_corpus(anandabazar, 1000)

In [5]:
filtered_ganashakti[:5], filtered_bartamanpatrika[:5],filtered_anandabazar[:5]

([[],
  ['আক্রান্ত', 'বিধান', 'বিপন্ন', 'গণতন্ত্র'],
  [],
  ['বিধান',
   'প্রস্তাব',
   'তন্ত্র',
   'শব্দ',
   'সরকার',
   'বেসরকারি',
   'হাত',
   'পরিষ্কার'],
  ['বিয়োজন',
   'চেয়ে',
   'বিজেপি',
   'সাংসদ',
   'সরকার',
   'মুখ',
   'এমন',
   'দেশ',
   'সামরিক']],
 [['শত',
   'নেতৃত্ব',
   'বিরাট',
   'ভারত',
   'ক্রিকেট',
   'কন্ট্রোল',
   'বোর্ড',
   'প্রস্তাব',
   'গ্রহণ',
   'শত',
   'টেস্ট',
   'অধিনায়ক',
   'বিদায়',
   'ম্যাচ',
   'মাইলস্টোন',
   'ম্যাচ',
   'কোন',
   'চাওয়া',
   'অভিমানী',
   'কে'],
  ['টেস্ট',
   'অধিনায়ক',
   'বিরাট',
   'কোকিল',
   'ইস্তফা',
   'পাঁচ',
   'বুমরাহকেও',
   'বেশ',
   'অবাক'],
  ['হ্যামস্ট্রিংয়ে',
   'চোট',
   'ওয়েটিং',
   'ইন্ডিজের',
   'বিরুদ্ধ',
   'সিরিজ',
   'রোহিত',
   'শর্করা',
   'ফেব্রুয়ারি',
   'গোড়ায়',
   'ভারত'],
  ['বিরাট', 'নতুন', 'অধিনায়ক', 'অধীনে', 'কাজ', 'কঠিন'],
  ['ের',
   'সপ্তম',
   'ব্যালট',
   'ডিও',
   'সি',
   'ল্যাং',
   'বা',
   'পায়ে',
   'জাদু',
   'হার']],
 [['১৫টা', 'লোক', 'ঘিরে'],
  ['ষষ্ঠী',
   'সপ্তম',
   '

In [19]:
def gen_words(texts):
    single_string = [ ' '.join(line) for line in texts]
    data_words = [gensim.utils.simple_preprocess(text) for text in single_string]
    return data_words
    
data_words  = gen_words(filtered_ganashakti)
print(data_words[9][:200])

['আরএসএস', 'অন', 'যবস', 'রহসন', 'পর', 'ণত']


In [20]:
# # CORPORA
id2word = corpora.Dictionary(data_words)
corpus = [id2word.doc2bow(text) for text in data_words]
print (corpus[0][0:20])

# word = id2word[[9][:1][0]]
# print (word)

[(0, 1), (1, 1), (2, 1)]


In [21]:
## BIGRAMMES ET TIGRAMMES

# # filtered_docs ou data_words
filtered_ganashakti = [list for list in filtered_ganashakti if len(list)>0]

# # list_of_docs = data_words

# bigram_phrases = gensim.models.Phrases(list_of_docs, min_count=5, threshold=50)
# trigram_phrases = gensim.models.Phrases(bigram_phrases[list_of_docs], threshold=50)

# bigram = gensim.models.phrases.Phraser(bigram_phrases)
# trigram = gensim.models.phrases.Phraser(trigram_phrases)

# # bigram, trigram

# def make_bigrams(texts):
#     # print(bigram[doc] for doc in texts)
#     return ([bigram[doc] for doc in texts])

# def make_trigrams(texts):
#     return([trigram[bigram[doc]] for doc in texts])


# data_bigrams = make_bigrams(list_of_docs)
# data_bigrams_trigrams = make_trigrams(data_bigrams)

data_bigrams_trigrams = make_bigrams_trigrams(filtered_bartamanpatrika)

In [22]:

print(data_bigrams_trigrams)

[['শত', 'নেতৃত্ব', 'বিরাট', 'ভারত', 'ক্রিকেট', 'কন্ট্রোল', 'বোর্ড', 'প্রস্তাব', 'গ্রহণ', 'শত', 'টেস্ট', 'অধিনায়ক', 'বিদায়', 'ম্যাচ', 'মাইলস্টোন', 'ম্যাচ', 'কোন', 'চাওয়া', 'অভিমানী', 'কে'], ['টেস্ট', 'অধিনায়ক', 'বিরাট', 'কোকিল', 'ইস্তফা', 'পাঁচ', 'বুমরাহকেও', 'বেশ', 'অবাক'], ['হ্যামস্ট্রিংয়ে', 'চোট', 'ওয়েটিং', 'ইন্ডিজের', 'বিরুদ্ধ', 'সিরিজ', 'রোহিত', 'শর্করা', 'ফেব্রুয়ারি', 'গোড়ায়', 'ভারত'], ['বিরাট', 'নতুন', 'অধিনায়ক', 'অধীনে', 'কাজ', 'কঠিন'], ['ের', 'সপ্তম', 'ব্যালট', 'ডিও', 'সি', 'ল্যাং', 'বা', 'পায়ে', 'জাদু', 'হার'], ['ভ্যাকসিন', 'নেব', 'নোভাক', 'জকোভিচ', 'প্রশ্ন', 'বিশ্ব'], ['ভার', 'গ্যালারি', 'নোনোভাক', 'লেখা', 'একা', 'পোস্ট', 'জকোভিচ', 'অস্ট্রেলিয়ান'], ['প্রত্যাখ্যাত', 'অ্যাথলেতিক', 'বিলাত', 'গোলে', 'সুপার', 'কাপ', 'ঘর', 'রিয়াল', 'মাদ্রাসা', 'রবিবার', 'সৌদি', 'আরব'], ['আইএসএলে', 'রানার', 'মোহন', 'বাগান', 'সুবাদ', 'ব্রিগেড', 'কাপের', 'প্লেঅফে']]


In [23]:
### TF-IDF
# from gensim.models import TfidfModel

# id2word = Dictionary(data_bigrams_trigrams)
# corpus = [id2word.doc2bow(text) for text in filtered_docs]

# print(corpus)

# tfidf = TfidfModel(corpus, id2word=id2word)
# low_value = 0.03

# words = []
# words_missing_in_tfidf = []

# # enlève les termes trops fréquents
# for i in range(0, len(corpus)):
#     bow = corpus[i]
#     low_value_words = [] #reinitialize to be safe. You can skip this.
#     tfidf_ids = [id for id, value in tfidf[bow]]
#     bow_ids = [id for id, value in bow]
#     low_value_words = [id for id, value in tfidf[bow] if value < low_value]
#     drops = low_value_words+words_missing_in_tfidf
#     for item in drops:
#         words.append(id2word[item])
#     words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

#     new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
#     corpus[i] = new_bow

In [32]:
corpus=get_corpus(filtered_ganashakti, data_bigrams_trigrams)
corpus[:20]

[[(0, 1), (1, 1), (2, 1), (3, 1)],
 [(2, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)],
 [(9, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1)],
 [(7, 1),
  (13, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1)],
 [(26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1)],
 [(2, 1),
  (16, 1),
  (30, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1)],
 [(8, 1),
  (13, 1),
  (36, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1)],
 [(13, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1)],
 [(36, 1),
  (44, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1)],
 [(63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1)]]

In [33]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=200,
                                           passes=10,
                                           alpha="auto")

IndexError: index 45 is out of bounds for axis 1 with size 45

In [34]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

IndexError: index 45 is out of bounds for axis 1 with size 45

#### Calcul spécificité

In [ ]:
def get_higest_lowest_specificity(filtered_docs, num=20):

    focus_dict = calculate_frequency(filtered_docs[:len(filtered_docs)//4])
    window_dict= calculate_frequency(filtered_docs[len(filtered_docs)//4:])
    
    print(focus_dict, len(focus_dict))
    print(window_dict, len(window_dict))
    
    print(f'len focus dict \t len window dict')
    print(len(focus_dict), len(window_dict))
    specificite = process_specif(focus_dict, window_dict, path="cache/") # dict specificite
    
    # affiche max et min
    max_values = sorted(specificite.items(), key = lambda x: x[1], reverse = True)[:num]
    min_values = sorted(specificite.items(), key = lambda x: x[1])[:num]
    
    print("\nMoins spécifiques :")
    for sp in min_values:
        print(sp)
        
    print("\nPlus spécifiques :")
    for sp in sorted(max_values, key= lambda x : x[1]): 
        print(sp)

    return max_values, min_values



In [ ]:
get_higest_lowest_specificity(filtered_bartamanpatrika)

In [ ]:
data = open(banglaGanashakti, 'r')
lines = data.readlines(1000) # lecture par ligne
lines = [line for line in lines if not line.startswith('Image')]
# print(lines[:10])
docs = [line.split() for line in lines]
docs[:10]

filtered_docs = filter_doc(docs)

for line, filtered_line in zip(docs[:5], filtered_docs[:5]):
    print(f'line = {line}, length = {len(line)}')
    print(f'filtered tokens = {filtered_line}, length = {len(filtered_line)}\n\n')

### Bartamanpatrika

In [ ]:
data = open(bartamanpatrika, 'r')
lines = data.readlines() # lecture par ligne
lines = [line for line in lines if not line.startswith('Image')]
# print(lines[:10])
docs = [line.split() for line in lines]
docs[:10]

filtered_docs = filter_doc(docs)

for line, filtered_line in zip(docs[:5], filtered_docs[:5]):
    print(f'line = {line}, length = {len(line)}')
    print(f'filtered tokens = {filtered_line}, length = {len(filtered_line)}\n\n')

In [ ]:
# ?
def gen_words(texts):
    data_words = [gensim.utils.simple_preprocess(text) for text in texts]
    return data_words

# single
single_string = [ ' '.join(line) for line in filtered_docs]
print(single_string[0][:100])
# for line in filtered_docs:
    
data_words  = gen_words(single_string)
print(data_words[0][:])



In [ ]:
# CORPORA
id2word = corpora.Dictionary(data_words)

corpus = [id2word.doc2bow(text) for text in data_words]
print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

In [ ]:
## BIGRAMMES ET TRIGRAMMES

# filtered_docs ou data_words
list_of_docs = [list for list in filtered_docs if len(list)>0]
# list_of_docs = data_words

bigram_phrases = gensim.models.Phrases(list_of_docs, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[list_of_docs], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

# bigram, trigram

def make_bigrams(texts):
    # print(bigram[doc] for doc in texts)
    return ([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return([trigram[bigram[doc]] for doc in texts])


data_bigrams = make_bigrams(list_of_docs)
data_bigrams_trigrams = make_trigrams(data_bigrams)

In [ ]:

print(data_bigrams_trigrams)

In [ ]:
### TF-IDF
from gensim.models import TfidfModel

id2word = Dictionary(data_bigrams_trigrams)
corpus = [id2word.doc2bow(text) for text in filtered_docs]

print(corpus)

tfidf = TfidfModel(corpus, id2word=id2word)
low_value = 0.03

words = []
words_missing_in_tfidf = []

# enlève les termes trops fréquents
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=50,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

### Anandabazar